In [2]:
#############################
# audio_processor.py
#############################
import os
import whisper
from moviepy.audio.io.AudioFileClip import AudioFileClip
from utils import time_to_seconds
import re
import pickle
CACHE_FILE = "processed_result_cache.pkl"



def transcribe_with_timestamps(audio_path):
    """Convert audio to text with word-level timestamps using Whisper"""
    print("loading speech to text model ....    ")
    #model = whisper.load_model("medium")
    print("speech to text model running ....    ")

    ####################Cache the processed audio####################
    # Check if cached audio exists
    if os.path.exists(CACHE_FILE):
        print("Loading cached processed_audio...")
        with open(CACHE_FILE, "rb") as f:
            result = pickle.load(f)
    else:
        print("Processing audio and caching the result...")
        model = whisper.load_model("medium")
        result = model.transcribe(audio_path, fp16=False, word_timestamps=True)

    # Save to cache
    with open(CACHE_FILE, "wb") as f:
        pickle.dump(result, f)
    ###############################################################       
    return result["segments"]

def align_script_with_audio(script_path, audio_segments):
    """
    Aligns the original script with the audio transcription.
    This function reads a script from a file and aligns it with the provided audio segments.
    It ensures that each word in the script corresponds to a word in the audio segments.
    Args:
        script_path (str): The file path to the script.
        audio_segments (list): A list of dictionaries, where each dictionary represents an audio segment
                               and contains 'words' (a list of word dictionaries with 'word', 'start', and 'end' keys),
                               'text' (the transcribed text of the segment), 'start' (start time of the segment),
                               and 'end' (end time of the segment).
    Returns:
        list: A list of dictionaries, where each dictionary contains:
              - 'script_line' (str): A line from the script.
              - 'words' (list): A list of word dictionaries with 'text', 'start', and 'end' keys.
              - 'start' (float): The start time of the segment.
              - 'end' (float): The end time of the segment.
    Raises:
        ValueError: If the number of words in the audio segments does not match the number of words in the script.
    """
    with open(script_path, encoding="utf-8") as f:
        script_lines = [line.strip() for line in f.readlines()]
    
    # converting audio_segments into list of words ####
    audio_words = []
    for segment in audio_segments:
        audio_words += segment['words']
    ######################################################
    script = {}
    script["script_lines"] = script_lines
    script["script_len"] = len(script_lines)
    script["script_line_words"] = []
    script["script_all_words"] = []
    for text in script_lines:
        text = re.sub(r'[^a-zA-Z\s]', '', text) #removing all special characters
        script["script_all_words"] += re.findall(r"\b\w+(?:'\w+)?\b", text)
        script["script_line_words"].append(re.findall(r"\b\w+(?:'\w+)?\b", text))
        # todo: change the text in evry word object in segment ( later)

    # checking the number of audio_words and script_lines words
    if ( len(audio_words) == len( script['script_all_words']) ):
        #iterate through every segment word and replace with script word
        word_ind = 0
        # audio_segments
        for idx,segment in enumerate(audio_segments):
            if idx >= script['script_len']:
                audio_segments = audio_segments[0:idx]
                break
            
            segment['text'] = script['script_lines'][idx]
            segment['words'] = audio_words[
                word_ind:len(script['script_line_words'][idx]) + word_ind
            ]
            word_ind += len(script['script_line_words'][idx])
            # for seg_word in  script['script_line_words'][idx] :
            #     seg_word = script['script_line_words'][word_ind]
            #     word_ind+=1
            segment['start'] = segment['words'][0]['start']
            segment['end'] = segment['words'][-1]['end']
        
        

    else:
        print("#########Number of words in audio and script are not equal#########3")
        return

    aligned_data = []
    for idx, segment in enumerate(audio_segments):
        if idx >= len(script_lines):
            break
        words = []
        for word in segment['words']:
            words.append({
                'text': word['word'],
                'start': word['start'],
                'end': word['end']
            })
        
        aligned_data.append({
            'script_line': script_lines[idx],
            'words': words,
            'start': segment['start'],
            'end': segment['end']
        })
    
    return aligned_data

def process_audio(audio_path, script_path):
    """Main audio processing function"""
    print("Processing audio...")
    audio_segments = transcribe_with_timestamps(audio_path)
    aligned_data = align_script_with_audio(script_path, audio_segments)
    print("audio process is completed")
    return {
        'raw_audio': AudioFileClip(audio_path),
        'aligned_data': aligned_data
    }


In [19]:
#############################
# video_processor.py (Updated for MoviePy 2.1.1)
#############################
from moviepy.video.VideoClip import ImageClip, TextClip
from moviepy.video.compositing.CompositeVideoClip import CompositeVideoClip
from moviepy.video.compositing.CompositeVideoClip import concatenate_videoclips
import os

def create_image_clips(image_dir, aligned_data):
    """Create image clips with proper sequencing and duration"""
    images = sorted([os.path.join(image_dir, f) for f in os.listdir(image_dir)], 
                   key=lambda x: int(os.path.splitext(os.path.basename(x))[0]))
    
    clips = []
    test_start = [0,4.56,9.82,6]
    test_durations = [4.2,4.58,2.44]
    for idx, segment in enumerate(aligned_data):
        if idx >= len(images):
            break
            
        # clip = ImageClip(images[idx]).with_start(test_start[idx],change_end=True)
        # clip = clip.with_duration(test_durations[idx])
        duration = segment['end'] - segment['start']        
        print(f"Segment {idx}: Start={segment['start']}, End={segment['end']}, Duration={duration}")
        clip = ImageClip(images[idx]).with_start(segment['start'],change_end=False)
        if(idx == len(aligned_data)-1):
            #final clip end with last segment end
            clip = clip.with_end(segment['end'])
        else:
            #clip end with next segment start
            clip = clip.with_end(aligned_data[idx+1]['start'])
        
        # clip = clip.with_duration(segment['end'] - segment['start'])
        # print(f"Clip {idx} duration after assignment: {clip.duration}")
        # print(f"Clip {idx} start  after assignment: {clip.start}")
        # print(f"Clip {idx} end  after assignment: {clip.end}")


        # clip.preview(fps=24)
        # print("clip duration")
        # print(idx)
        # print(clip.duration)
        clips.append(clip)  #set_start(segment['start']))
    
    return concatenate_videoclips(clips, method="chain")#"chain")

def create_subtitles(aligned_data, sub_position):
    """Generate animated word-level subtitles"""
    subtitle_clips = []
    font_name = r"fonts\Arial.otf"
    for segment in aligned_data:
        words = segment['words']
        
        for word in words:
            txt_clip = TextClip(
                text=word['text'],
                font_size=70,
                font=font_name,
                color='white',
                stroke_color='black',
                stroke_width=2
            ).with_start(word['start']).with_duration(word['end'] - word['start'])
            
            txt_clip = txt_clip.with_position(('center', sub_position))
            # txt_clip.preview(fps=24)
            subtitle_clips.append(txt_clip)
    
    return subtitle_clips

def process_video(image_dir, script_path, audio_data, output_path, sub_position):
    """Main video processing function"""
    video_clip = create_image_clips(image_dir, audio_data['aligned_data'])
    # video_clip.preview(fps=24)
    subtitles = create_subtitles(audio_data['aligned_data'], 'center') #sub_position)#change here
    
    final_video = CompositeVideoClip([video_clip] + subtitles)
    # final_video.preview(fps=24)
    final_video = final_video.with_audio(audio_data['raw_audio'])
    # final_video = final_video.with_speed_scaled(1.5)
    # final_video.preview(fps=24)
    
    final_video.write_videofile(
        output_path,
        codec='libx264',
        audio_codec='aac',
        fps=24,
        threads=4,
        preset='fast'
    )




In [20]:
image_dir = "Test_data/images"
script_path = "Test_data/dronacharya killed/Script/script.txt"
audio_path = "Test_data/dronacharya killed/audio/script.mp3"
output = "final_video.mp4"
sub_pos = 30

processed_audio = process_audio(audio_path, script_path)
process_video(image_dir, script_path, processed_audio,
                output, sub_pos)


Processing audio...
loading speech to text model ....    
speech to text model running ....    
Loading cached processed_audio...
audio process is completed
Segment 0: Start=0.0, End=4.2, Duration=4.2
Segment 1: Start=4.56, End=9.14, Duration=4.580000000000001
Segment 2: Start=9.82, End=12.26, Duration=2.4399999999999995
Segment 3: Start=12.26, End=15.12, Duration=2.8599999999999994
Segment 4: Start=16.16, End=21.08, Duration=4.919999999999998
Segment 5: Start=21.54, End=28.8, Duration=7.260000000000002
Segment 6: Start=29.52, End=34.74, Duration=5.220000000000002
Segment 7: Start=35.38, End=41.02, Duration=5.640000000000001
Segment 8: Start=41.78, End=46.0, Duration=4.219999999999999
MoviePy - Building video final_video.mp4.
MoviePy - Writing audio in final_videoTEMP_MPY_wvf_snd.mp4


MoviePy - Done.
MoviePy - Writing video final_video.mp4



MoviePy - Done !
MoviePy - video ready final_video.mp4


In [21]:
import os
import whisper
from moviepy.audio.io.AudioFileClip import AudioFileClip
from utils import time_to_seconds
import re
import pickle
import tempfile

In [22]:
with tempfile.NamedTemporaryFile(suffix=".wav", delete=False) as temp_file:
    temp_path = temp_file.name

In [26]:
audio_path = "Test_data/dronacharya killed/audio/script.mp3"
original_audio = AudioFileClip(audio_path)
sped_up_audio = original_audio.with_speed_scaled(1.25) #speedx(factor=1.25)
sped_up_audio.write_audiofile(temp_path)

MoviePy - Writing audio in C:\Users\ajitp\AppData\Local\Temp\tmphsk2kzg6.wav


MoviePy - Done.


In [2]:
import os
from moviepy.editor import VideoFileClip
from pydub import AudioSegment

def speed_up_video(input_file, output_file, speed_factor):
    # Extract audio from video
    temp_audio = "temp_audio.wav"
    os.system(f'ffmpeg -i "{input_file}" -q:a 0 -map a "{temp_audio}" -y')

    # Apply time stretching using rubberband
    processed_audio = "processed_audio.wav"
    os.system(f"rubberband -t {speed_factor} {temp_audio} {processed_audio}")

    # Speed up the video without audio
    temp_video = "temp_video.mp4"
    os.system(f'ffmpeg -i "{input_file}" -filter:v "setpts={1/speed_factor}*PTS" "{temp_video}" -y')

    # Merge new audio with video
    os.system(f'ffmpeg -i "{temp_video}" -i "{processed_audio}" -c:v copy -c:a aac -strict experimental "{output_file}" -y')

    # Cleanup temp files
    os.remove(temp_audio)
    os.remove(processed_audio)
    os.remove(temp_video)

    print(f"Processed video saved as: {output_file}")

# Example Usage:
input_video = "input.mp4"
output_video = "output.mp4"
speed_factor = 1.5  # Increase speed by 1.5x

speed_up_video(input_video, output_video, speed_factor)


ModuleNotFoundError: No module named 'librosa'

In [1]:
import librosa
import soundfile as sf

def increase_speed_without_pitch_change_librosa(input_file, output_file, speed_factor=1.25):
    """
    Increase the speed of an audio file without changing its pitch using librosa.
    
    Args:
        input_file (str): Path to the input audio file.
        output_file (str): Path to save the output audio file.
        speed_factor (float): Speed multiplier (e.g., 1.25 for 1.25x speed).
    """
    # Load the audio file
    y, sr = librosa.load(input_file, sr=None)
    
    # Time-stretch the audio
    y_stretched = librosa.effects.time_stretch(y, rate=speed_factor)
    
    # Save the result
    sf.write(output_file, y_stretched, sr)
    print(f"Audio saved to {output_file}")

# Example usage
input_file = "Test_data\aswathama killed\audio\script.mp3"  # Replace with your input file path
output_file = "voiceover_fast_librosa.wav"  # Replace with your desired output file path
increase_speed_without_pitch_change_librosa(input_file, output_file, speed_factor=1.25)

<>:24: SyntaxWarning: invalid escape sequence '\s'
<>:24: SyntaxWarning: invalid escape sequence '\s'
C:\Users\ajitp\AppData\Local\Temp\ipykernel_15776\3346408553.py:24: SyntaxWarning: invalid escape sequence '\s'
  input_file = "Test_data\aswathama killed\audio\script.mp3"  # Replace with your input file path
C:\Users\ajitp\AppData\Local\Temp\ipykernel_15776\3346408553.py:24: SyntaxWarning: invalid escape sequence '\s'
  input_file = "Test_data\aswathama killed\audio\script.mp3"  # Replace with your input file path


ModuleNotFoundError: No module named 'librosa'

In [30]:
from pydub import AudioSegment
from pydub.playback import play
import os

def increase_speed_without_pitch_change(input_file, output_file, speed_factor=1.5):
    """
    Increase the speed of an audio file without changing its pitch.
    
    Args:
        input_file (str): Path to the input audio file.
        output_file (str): Path to save the output audio file.
        speed_factor (float): Speed multiplier (e.g., 1.25 for 1.25x speed).
    """
    # Load the audio file
    audio = AudioSegment.from_file(input_file)
    
    # Time-stretch the audio (speed up without changing pitch)
    stretched_audio = audio.speedup(playback_speed=speed_factor)
    
    # Export the result
    stretched_audio.export(output_file, format="wav")
    print(f"Audio saved to {output_file}")

# Example usage
input_file = "Test_data/dronacharya killed/audio/script.mp3"  # Replace with your input file path
output_file = "voiceover_fast.wav"  # Replace with your desired output file path
increase_speed_without_pitch_change(input_file, output_file, speed_factor=1.3)

Audio saved to voiceover_fast.wav


In [ ]:
from moviepy.video.fx.all import speedx  # Import the speedx effect

def process_video(image_dir, script_path, audio_data, output_path, sub_position, speed_factor=1.0):
    """Main video processing function with speed change applied only to the final composite clip."""
    # Create the video clip and subtitles as before (no speed changes here)
    video_clip = create_image_clips(image_dir, audio_data['aligned_data'])
    subtitles = create_subtitles(audio_data['aligned_data'], sub_position)
    
    # Composite the video clip with the subtitles and audio
    final_video = CompositeVideoClip([video_clip] + subtitles)
    final_video = final_video.with_audio(audio_data['raw_audio'])
    
    # Apply the speed change to the final composite video clip only
    final_video = final_video.fx(speedx, speed_factor)
    
    # Write out the final video file (use a fixed fps or adjust if needed)
    final_video.write_videofile(
        output_path,
        codec='libx264',
        audio_codec='aac',
        fps=24,
        threads=4,
        preset='fast'
    )


ModuleNotFoundError: No module named 'moviepy.video.fx.all'

In [14]:
from moviepy.video.fx.all import speedx  # Import the speedx effect

def process_video(image_dir, script_path, audio_data, output_path, sub_position, speed_factor=1.0):
    """Main video processing function with speed change applied only to the final composite clip."""
    # Create the video clip and subtitles as before (no speed changes here)
    video_clip = create_image_clips(image_dir, audio_data['aligned_data'])
    subtitles = create_subtitles(audio_data['aligned_data'], sub_position)
    
    # Composite the video clip with the subtitles and audio
    final_video = CompositeVideoClip([video_clip] + subtitles)
    final_video = final_video.with_audio(audio_data['raw_audio'])
    
    # Apply the speed change to the final composite video clip only
    final_video = final_video.fx(speedx, speed_factor)
    
    # Write out the final video file (use a fixed fps or adjust if needed)
    final_video.write_videofile(
        output_path,
        codec='libx264',
        audio_codec='aac',
        fps=24,
        threads=4,
        preset='fast'
    )


ModuleNotFoundError: No module named 'moviepy.video.fx.all'